In [2]:
import cv2
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def extract_features(crop):
    resized = cv2.resize(crop, (64, 128))
    hsv = cv2.cvtColor(resized, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1, 2], None, [8, 8, 8], [0, 180, 0, 256, 0, 256])
    hist = cv2.normalize(hist, hist).flatten()
    return hist

def match_players(feats1, feats2, threshold=0.8):
    ids1 = list(feats1.keys())
    ids2 = list(feats2.keys())
    X = np.array([feats1[i] for i in ids1])
    Y = np.array([feats2[i] for i in ids2])

    sim = cosine_similarity(X, Y)
    matches = {}

    for i, row in enumerate(sim):
        max_idx = np.argmax(row)
        if row[max_idx] > threshold:
            matches[ids1[i]] = ids2[max_idx]
    return matches

def draw_player_boxes(frame, boxes, ids):
    for box, pid in zip(boxes, ids):
        x1, y1, x2, y2 = map(int, box)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"ID: {pid}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
    return frame